In [1]:
import numpy as np
from CoolProp.CoolProp import PropsSI


### Specify Parameters

In [2]:
# tsp=thermosyphon
Nt=9
l_e= 0.45 # evaporator tsp len
l_a=0.1 # adiabatic tsp len
l_c=0.45 # condenser tsp len
tsp_od=9.5/1000
tsp_thick=1/1000
PT=15/1000 #tube pitch
F=70/100 #filling ratio


B=86/1000 #baffle spacing
shell_inlet_dia=25.4/1000
shell_id=99.6/1000

m = 4/60 #  L/min to kg/s
Ti_e=100 #°C
Ti_c=20 #°C
To_e=60 # assume
To_c=40 # assume
q_hx=5000
q_e=q_hx #heat flux
q_c=q_hx
k_ss=16.2 # W/mK
k_ms=50 

### Calculate Primary parameters

In [3]:
tsp_id=tsp_od-tsp_thick*2
#hydraulic dia
Dhs=4*PT**2/(np.pi*tsp_od)-tsp_od
#Shell inside dia
Ds=shell_id
#Clearance
C=PT-tsp_od
#Bundle cross flow area
As=Ds*C*B/PT




In [4]:
Cp_e = PropsSI('C', 'T', Ti_e + 273.15, 'P', 101325, 'Water')  # Specific heat capacity (J/(kg·K))
Cp_c = PropsSI('C', 'T', Ti_c + 273.15, 'P', 101325, 'Water')  # Specific heat capacity (J/(kg·K))

Tv=(Ti_e*Cp_e+Ti_c*Cp_c)/(Cp_e+Cp_c)
print(f"Vapor Temperature, Tv = {Tv}")

Vapor Temperature, Tv = 46.56270354518455


### Evaporator

In [5]:
fluid = "Water"
temperature = Tv + 273.15  # Convert °C to K
pressure = 101325  # Pressure in Pa (1 atm)

#properties
rho_l = PropsSI('D', 'T', temperature, 'P', pressure, fluid)  # Density in kg/m³
rho_v = PropsSI('D', 'T', temperature, 'Q', 1, fluid)  # Density in kg/m³ (Q=1 means saturated vapor)
mu = PropsSI('V', 'T', temperature, 'P', pressure, fluid)  # Dynamic viscosity (Pa·s)
Cp = PropsSI('C', 'T', temperature, 'P', pressure, fluid)  # Specific heat capacity (J/(kg·K))
k = PropsSI('L', 'T', temperature, 'P', pressure, fluid)  # Thermal conductivity (W/(m·K))
Pr=PropsSI('PRANDTL', 'T', temperature, 'P', pressure, fluid) # Prandtl Number
P_v = PropsSI('P', 'T', temperature, 'Q', 0, fluid)  # 'Q=0' means saturated liquid
h_lv = PropsSI('H', 'T', temperature, 'Q', 1, fluid) - PropsSI('H', 'T', temperature, 'Q', 0, fluid)



print(f"Properites of {fluid} at {Tv} °C:")
print(f"\tρ_l = {rho_l} kg/m³")
print(f"\tρ_v = {rho_v} kg/m³")
print(f"\tμ = {mu} Pa·s")
print(f"\tPr = {Pr}")
print(f"\tk = {k}")
print(f"Vapor Pressure, Pv = {P_v}")
print(f"Latent heat of vaporization, h_lv = {h_lv}")



Properites of Water at 46.56270354518455 °C:
	ρ_l = 989.5504498046275 kg/m³
	ρ_v = 0.07068439736441362 kg/m³
	μ = 0.0005796176375624808 Pa·s
	Pr = 3.805937637132946
	k = 0.6366562477141507
Vapor Pressure, Pv = 10393.187046845764
Latent heat of vaporization, h_lv = 2390233.2624841817


In [6]:
Vs=m/(rho_l*As)
print(f"Evaporator Shell Side Velocity, Vs = {Vs}")

Re_s=Vs*Dhs*rho_l/mu
print(f"Renolds Number, Re_s = {Re_s}")

Nu = 0.36*pow(Re_s,0.55)*pow(Pr,1/3)
print(f"Nusselt Number, Nu = {Nu}")

#coefficient of external heat transer in evaporator
h_ex_e=Nu*k/Dhs
print(f"Heat transfer coefficient, h_ex_e = {h_ex_e}")
A_e=np.pi*tsp_od*l_e*Nt
# Convection resistance in hot fluid
R1=1/(h_ex_e*A_e)
print(f"Forced convection resistance over Evaporator, R1 = {R1}")

r_e=tsp_od/2
r_i=r_e-tsp_thick
R2=np.log(r_e/r_i)/(2*np.pi*k_ss*l_e)
print(f"Radial heat conduction resistance through Evaporator wall, R2 = {R2}")

phi2=pow(h_lv*k*(rho_l**0.5)/mu,0.25)*(pow(rho_l,0.65)*pow(k,0.3)*pow(Cp,0.7))/(pow(rho_v,0.25)*pow(h_lv,0.4)*pow(mu,0.7))
phi3=pow(P_v/101325,0.23)*(pow(rho_l,0.65)*pow(k,0.3)*pow(Cp_e,0.7))/(pow(rho_v,0.25)*pow(h_lv,0.4)*pow(mu,0.7))
#thermal resistance associated to pool boiling
R3p=1/((9.81**0.2)*phi3*(q_e**0.4)*pow(np.pi*tsp_id*l_e,0.6))
#thermal resistance associated to falling film located above the evaporator pool
R3f=0.345*pow(q_e,1/3)/(pow(tsp_id,4/3)*pow(9.81,1/3)*l_e*pow(phi2,4/3))
R3=R3p*F+(1-F)*R3f
print(f"Boiling thermal resistance, R3 = {R3}")

R_e=R1+R2+R3
print(f"Evaporator Resistance, R_e = {R_e}")

Evaporator Shell Side Velocity, Vs = 0.02145070549348493
Renolds Number, Re_s = 756.4448788945504
Nusselt Number, Nu = 21.53362924164969
Heat transfer coefficient, h_ex_e = 663.716902791145
Forced convection resistance over Evaporator, R1 = 0.012464894896431341
Radial heat conduction resistance through Evaporator wall, R2 = 0.005160828877966428
Boiling thermal resistance, R3 = 2.3613973322893273e-05
Evaporator Resistance, R_e = 0.017649337747720664


### Condenser

In [7]:
fluid = "Water"
temperature = Tv + 273.15  # Convert °C to K
pressure = 101325  # Pressure in Pa (1 atm)

#properties
rho_l = PropsSI('D', 'T', temperature, 'P', pressure, fluid)  # Density in kg/m³
rho_v = PropsSI('D', 'T', temperature, 'Q', 1, fluid)  # Density in kg/m³ (Q=1 means saturated vapor)
mu = PropsSI('V', 'T', temperature, 'P', pressure, fluid)  # Dynamic viscosity (Pa·s)
Cp_c = PropsSI('C', 'T', temperature, 'P', pressure, fluid)  # Specific heat capacity (J/(kg·K))
k = PropsSI('L', 'T', temperature, 'P', pressure, fluid)  # Thermal conductivity (W/(m·K))
Pr=PropsSI('PRANDTL', 'T', temperature, 'P', pressure, fluid) # Prandtl Number
P_v = PropsSI('P', 'T', temperature, 'Q', 0, fluid)  # 'Q=0' means saturated liquid
h_lv = PropsSI('H', 'T', temperature, 'Q', 1, fluid) - PropsSI('H', 'T', temperature, 'Q', 0, fluid)



print(f"Properites of {fluid} at {Tv} °C:")
print(f"\tρ_l = {rho_l} kg/m³")
print(f"\tρ_v = {rho_v} kg/m³")
print(f"\tμ = {mu} Pa·s")
print(f"\tPr = {Pr}")
print(f"\tk = {k}")
print(f"Vapor Pressure, Pv = {P_v}")
print(f"Latent heat of condensation, h_lv = {h_lv}")



Properites of Water at 46.56270354518455 °C:
	ρ_l = 989.5504498046275 kg/m³
	ρ_v = 0.07068439736441362 kg/m³
	μ = 0.0005796176375624808 Pa·s
	Pr = 3.805937637132946
	k = 0.6366562477141507
Vapor Pressure, Pv = 10393.187046845764
Latent heat of condensation, h_lv = 2390233.2624841817


In [8]:
Vs=m/(rho_l*As)
print(f"Condenser Shell Side Velocity, Vs = {Vs}")

Re_s=Vs*Dhs*rho_l/mu
print(f"Renolds Number, Re_s = {Re_s}")

Nu = 0.36*pow(Re_s,0.55)*pow(Pr,1/3)
print(f"Nusselt Number, Nu = {Nu}")

#coefficient of external heat transer in evaporator
h_ex_c=Nu*k/Dhs
print(f"Convection Heat transfer coefficient, h_ex_c = {h_ex_c}")
A_c=np.pi*tsp_od*l_c*Nt
# Convection resistance in hot fluid
R9=1/(h_ex_c*A_c)
print(f"Forced convection resistance over Condenser, R9 = {R9}")

r_e=tsp_od/2
r_i=r_e-tsp_thick
R8=np.log(r_e/r_i)/(2*np.pi*k_ss*l_c)
print(f"Radial heat conduction resistance through Condenser wall, R8 = {R8}")

phi2=pow(h_lv*k*(rho_l**0.5)/mu,0.25)*(pow(rho_l,0.65)*pow(k,0.3)*pow(Cp_c,0.7))/(pow(rho_v,0.25)*pow(h_lv,0.4)*pow(mu,0.7))
R7=0.345*pow(q_c,1/3)/(pow(tsp_id,4/3)*pow(9.81,1/3)*l_c*pow(phi2,4/3))
print(f"Condensation thermal resistance, R7 = {R7}")

R_c=R1+R2+R3
print(f"Condenser Resistance, R_c = {R_c}")


Condenser Shell Side Velocity, Vs = 0.02145070549348493
Renolds Number, Re_s = 756.4448788945504
Nusselt Number, Nu = 21.53362924164969
Convection Heat transfer coefficient, h_ex_c = 663.716902791145
Forced convection resistance over Condenser, R9 = 0.012464894896431341
Radial heat conduction resistance through Condenser wall, R8 = 0.005160828877966428
Condensation thermal resistance, R7 = 1.2053343649631534e-06
Condenser Resistance, R_c = 0.017649337747720664


In [9]:
R_total=R_e+R_c
print(f"Total thermal resistance, R_total = {R_total}")

Total thermal resistance, R_total = 0.03529867549544133


In [10]:
q_th=q_hx/Nt
Tw_e = Tv + (R2+R3)*q_th
Tw_c = Tv - (R7+R9)*q_th
Tavg_e=Tv-(((Tv-Ti_e)-(Tv-To_e))/np.log((Tv-Ti_e)/(Tv-To_e)))
Tavg_c=Tv-(((Tv-Ti_c)-(Tv-To_c))/np.log((Tv-Ti_c)/(Tv-To_c)))
q_e=(Tavg_e-Tv)/R_e
q_c=(Tavg_c-Tv)/R_c

## Optimization 

In [11]:
#tolerance
alpha=abs(1-q_c/q_e)
alpha

1.493694793888157

In [12]:
i=0
while(alpha>0.1/100 and i<50):
    Tv=(R_c*Tavg_e+R_e*Tavg_c)/(R_e+R_c)
    q_hx=(Tavg_e-Tavg_c)/R_total
    q_th=q_hx/Nt
    Tw_e = Tv + (R2+R3)*q_th
    Tw_c = Tv - (R7+R9)*q_th
    Tavg_e=Tv-(((Tv-Ti_e)-(Tv-To_e))/np.log((Tv-Ti_e)/(Tv-To_e)))
    Tavg_c=Tv-(((Tv-Ti_c)-(Tv-To_c))/np.log((Tv-Ti_c)/(Tv-To_c)))
    q_e=(Tavg_e-Tv)/R_e
    q_c=(Tv-Tavg_c)/R_c
    alpha=abs(1-q_c/q_e)
    print(f'iteration {i}: α = {alpha*100}% q_hx = {q_hx}')
    i+=1





iteration 0: α = 13.402843162616485% q_hx = 1226.1256341927294
iteration 1: α = 2.6221692701912325% q_hx = 1195.8345011974961
iteration 2: α = 0.5396020394748291% q_hx = 1206.3969456325203
iteration 3: α = 0.10986409791885476% q_hx = 1204.385391852789
iteration 4: α = 0.02241690136608465% q_hx = 1204.8019667542706


In [14]:
To_e_new=Ti_e-q_hx/(m*Cp_e)
To_c_new=Ti_c+q_hx/(m*Cp_c)


In [16]:

print(f"Evaporator Outlet Temperature: {To_e_new}")

print(f"Condensor Outlet Temperature: {To_c_new}")

Evaporator Outlet Temperature: 91.31075249967779
Condensor Outlet Temperature: 24.32296626235573
